In [65]:
import numpy as np
import pandas as pd

## Load Data

In [66]:
# Define the constants for the scenario
pilot = 'PILOT'
flight = 'FLIGHT'
lead_alt = 15000
wing_alt = 10000
CM_Airspeed = 150

In [67]:
# Load all data sources from a given flight
lead_data = pd.read_csv('Data/Lead/Lead_{}_{}.csv'.format(pilot, flight))
wing_data = pd.read_csv('Data/Wingman/Wing_{}_{}.csv'.format(pilot, flight))
cruise_data = pd.read_csv('Data/CruiseMissiles/CMs_{}_{}.csv'.format(pilot, flight))
workload_data = pd.read_csv('Data/Workload/Workload_{}_{}.csv'.format(pilot, flight))
# comm_data = pd.read_csv('Data/Comm/Comms_{}_{}.csv'.format(pilot, flight))
# surface_data = pd.read_csv('Data/SurfaceThreats/SurfaceThreats_{}_{}.csv'.format(pilot, flight))

## Define the Master DF

In [68]:
Altitude_Col = 'Altitude_Msl'
Airpseed_Col = 'True_Airspeed'
cols_L29s = ['Timestamp', 'SampleDate', 'SampleTime', 'TestCard', 'True_Heading', 'Roll',
        'Latitude', 'Longitude', 'Vertical_Speed', Altitude_Col, Airpseed_Col]
cols_CMs = ['Timestamp', 'SampleDate', 'SampleTime', 'TestCard', 'DisSource',
            'DisTime', 'Aplication', 'EntId', 'Latitude', 'Longitude']

In [71]:
# Make sure Timestamp is sorted and in datetime format
lead_data['Timestamp'] = pd.to_datetime(lead_data['Timestamp'])
wing_data['Timestamp'] = pd.to_datetime(wing_data['Timestamp'])
cruise_data['Timestamp'] = pd.to_datetime(cruise_data['Timestamp'])

lead_data = lead_data.sort_values('Timestamp')
wing_data = wing_data.sort_values('Timestamp')
cruise_data = cruise_data.sort_values('Timestamp')

# Merge wing onto lead (treat lead as "truth")
# This is necessary since the lead aircraft is the primary source of data and the timest stamps may not match perfectly.
sortie_df = pd.merge_asof(
    lead_data[cols_L29s].sort_values('Timestamp'),
    wing_data[cols_L29s].sort_values('Timestamp'),
    on='Timestamp',
    direction='nearest',
    suffixes=('_Lead', '_Wing'),
    tolerance=pd.Timedelta('50ms')  # adjust tolerance as needed
)

# Merge cruise missiles the same way
sortie_df = pd.merge_asof(
    sortie_df,
    cruise_data[cols_CMs].sort_values('Timestamp'),
    on='Timestamp',
    direction='nearest',
    suffixes=('', '_CM'),
    tolerance=pd.Timedelta('50ms')
)

In [72]:
# add Cruise Missile meta data to sortie_df
sortie_df['CM_Altitude_Lead'] = lead_alt
sortie_df['CM_Altitude_Wing'] = wing_alt
sortie_df['CM_Airspeed'] = CM_Airspeed

## Begin Generating MOPs